# model11 : copia do modelo03, mas com melhor eff net

efficien net com todas as features + data augmentation e é só

In [87]:
import pandas as pd

In [88]:
df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

In [89]:
set(df_train["img_placement"]) | set(df_val["img_placement"])

{'S-unknown', 'openspace', 'r_openspace', 'roof'}

In [90]:
set(df_train["img_origin"]) | set(df_val["img_origin"])

{'D', 'S'}

In [91]:
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["img_placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["img_placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["img_placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["img_placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["img_placement"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [92]:
df_train.sample(3)

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
1237,IDuPhRi8,[],[],[1],"[[(220.544, 277.3333333333333), (266.88, 279.8...","[1, 0, 0, 0, 1, 0]"
1250,IDzXGb171Wn,[],[],[1],"[[(204.672, 196.77866666666668), (219.904, 208...","[1, 0, 0, 0, 1, 0]"
93,ID3FeIVccpCuIM,[],[],"[1, 1, 1]","[[(263.168, 293.5466666666667), (250.368, 314....","[1, 0, 0, 0, 1, 0]"


---

## efficient net v2

In [93]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import AdamW
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import Sequence

In [94]:
class ImageDataGeneratorCustom(tf.keras.utils.Sequence):
    def __init__(self, df, image_size=(512, 512), batch_size=16, shuffle=True, augment=False):
        self.df = df
        self.image_size = image_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.indexes = self.df.index.values
        self.on_epoch_end()

        if self.augment:
            self.transform = A.Compose([
                A.RandomResizedCrop((self.image_size[0], self.image_size[1]), scale=(0.7, 1.0), ratio=(0.75, 1.33)),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomRotate90(p=0.5),
                A.GaussianBlur(blur_limit=(3, 7), p=0.3),
                A.CLAHE(clip_limit=4.0, p=0.5),
                A.HueSaturationValue(p=0.3),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2()
            ])
        else:
            # If no augmentation, only resize and normalize
            self.transform = A.Compose([
                A.Resize(self.image_size[0], self.image_size[1]),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2()
            ])

    def __len__(self):
        # Calculate the number of batches per epoch
        return int(np.floor(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch if shuffle is True
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
            #np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        # Get batch of data
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indexes]

        images = []
        labels = []
        extra_features = []

        for i, row in batch_df.iterrows():
            # Load image
            image_path = "images/" + row['img_id'] + '.jpg'
            img = load_img(image_path, target_size=self.image_size)
            img = img_to_array(img)  # Convert to numpy array
            img = img / 255.0 # Normalize pixel values
            
            img = self.transform(image=img)["image"]

            if img.shape[0] == 3:  # If it's in channels-first format (3, 512, 512)
                img = np.transpose(img, (1, 2, 0))

            
            images.append(img)
            labels.append((sum(row["nr_boil"]), sum(row["nr_pan"])))
            extra_features.append(row["more_features"])

        return (np.array(images), np.array(extra_features)), np.array(labels)


In [95]:
train_generator = ImageDataGeneratorCustom(df_train, image_size=(512, 512), batch_size=16, shuffle=True, augment=True)
val_generator = ImageDataGeneratorCustom(df_val, image_size=(512, 512), batch_size=16, shuffle=False, augment=False)

In [96]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [97]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate, LayerNormalization, MultiHeadAttention, Softmax
from tensorflow.keras.activations import linear
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization

# Define image input
image_input = Input(shape=(512, 512, 3), name="image_input")

# Define extra features input (6 additional features)
extra_input = Input(shape=(6,), name="extra_features")

# Meta processor for extra features
meta_processor = tf.keras.Sequential([
    Dense(128, input_dim=6, activation='relu'),
    LayerNormalization(),
    Dropout(0.3),
    Dense(64)
])

# Process extra features through the meta_processor
meta_features = meta_processor(extra_input)

# Base model (EfficientNetB5 without the top layers)
base_model = ResNet50V2(weights="imagenet", include_top=False, input_tensor=image_input)
for layer in base_model.layers[-3:]:  
    layer.trainable = True


# Feature extraction from CNN
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce spatial dimensions

# Merge image features with extra input (processed via meta_processor)
x = Concatenate()([x, meta_features])  # Combine CNN features with processed extra features

# Dense layers
x = Dense(512, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(64, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Output layer
output = Dense(2)(x)  # Regression output (2 values)
output = linear(output) 

# Define and compile the model
model = Model(inputs=[image_input, extra_input], outputs=output)

model.compile(
    optimizer=AdamW(),
    loss=["mae"],
    metrics=["mae"]
)

/opt/anaconda3/envs/CAA01/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [98]:
#model.summary()

In [99]:
from tensorflow.keras import callbacks

# Define ModelCheckpoint to save model after each epoch
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=f"ModelBuild/Model11_checkpoint.keras",
    save_weights_only=False,
    save_best_only=True,
    monitor="val_mae", 
    verbose=1
)

# Early stopping to prevent overfitting
early_stopping_callback = callbacks.EarlyStopping(
    monitor="val_mae",
    patience=5,
    restore_best_weights=True
)

history = model.fit(train_generator,
          validation_data=val_generator,
          epochs=25,
          callbacks=[checkpoint_callback, early_stopping_callback])

model.save('ModelBuild/model11.keras')

/opt/anaconda3/envs/CAA01/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 2.9415 - mae: 2.9415
Epoch 1: val_mae improved from inf to 37.98006, saving model to ModelBuild/Model11_checkpoint.keras
152/152 ━━━━━━━━━━━━━━━━━━━━ 815s 5s/step - loss: 2.9393 - mae: 2.9393 - val_loss: 37.9801 - val_mae: 37.9801
Epoch 2/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 2.2264 - mae: 2.2264
Epoch 2: val_mae improved from 37.98006 to 7.31513, saving model to ModelBuild/Model11_checkpoint.keras
152/152 ━━━━━━━━━━━━━━━━━━━━ 777s 5s/step - loss: 2.2268 - mae: 2.2268 - val_loss: 7.3151 - val_mae: 7.3151
Epoch 3/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 2.2733 - mae: 2.2733
Epoch 3: val_mae did not improve from 7.31513
152/152 ━━━━━━━━━━━━━━━━━━━━ 827s 5s/step - loss: 2.2729 - mae: 2.2729 - val_loss: 9013.3203 - val_mae: 9013.3203
Epoch 4/25
129/152 ━━━━━━━━━━━━━━━━━━━━ 2:01 5s/step - loss: 3.1406 - mae: 3.1406

KeyboardInterrupt: 

v01: adamw lr = learning_rate=0.00014 eeeee softmax output andddd EfficientNetB5 - *3.01*

v02: adamw , output linear, ResNet50V2 sem freeze nas ultiams 5 layers - *2.97*

In [ ]:
import json

a = history.history
with open('ModelBuild/model11_history.json', 'w') as f:
    json.dump(a, f)

with open('ModelBuild/model11_history.json') as f:
    data = json.load(f)

In [ ]:
plt.plot(data['loss'], label='Training Loss')
plt.plot(data['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(data['mae'], label='Training MAE')
plt.plot(data['val_mae'], label='Validation MAE')
plt.title('Training vs Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

# submission

In [ ]:
a = 

In [ ]:
model = tf.keras.models.load_model('ModelBuild/Model11_v02.keras')

In [ ]:
teste = pd.read_csv("Test.csv")
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["placement"], inplace=True)

hot_encod_img_placement(teste)
teste.rename(columns={"ID": "img_id"}, inplace=True)
submission = pd.read_csv("zindi_files/SampleSubmission.csv")

transform = A.Compose([
                A.Resize(512, 512),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2()
            ])

for i, row in teste.iterrows():
    image_path = "images/" + row['img_id'] + '.jpg'
    img = load_img(image_path, target_size=(512, 512))
    img = img_to_array(img)  # Convert to numpy array
    img = img / 255.0 # Normalize pixel values
    img = transform(image=img)["image"]

    if img.shape[0] == 3:  # If it's in channels-first format (3, 512, 512)
        img = np.transpose(img, (1, 2, 0))

    img = np.expand_dims(img, axis=0)

    extra_features = np.array(row["more_features"]).reshape(1, 6)

    prediction = model.predict([img, extra_features], verbose=0)
    pred_nr_boil = prediction[0][0]
    pred_br_pan = prediction[0][1]
    submission.loc[submission['ID'] == f"{row['img_id']}_boil", "Target"] = pred_nr_boil
    submission.loc[submission['ID'] == f"{row['img_id']}_pan", "Target"] = pred_br_pan

    if i % 100 == 0:
        print(f"{i} images processed of {len(teste)}")

/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_90586/687877350.py:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.040407050400972366' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission.loc[submission['ID'] == f"{row['img_id']}_boil", "Target"] = pred_nr_boil


0 images processed of 1107
100 images processed of 1107
200 images processed of 1107
300 images processed of 1107
400 images processed of 1107
500 images processed of 1107
600 images processed of 1107
700 images processed of 1107
800 images processed of 1107
900 images processed of 1107
1000 images processed of 1107
1100 images processed of 1107


In [ ]:
submission.head(20)

,ID,Target
0,ID00qprY_boil,-0.040407
1,ID00qprY_pan,2.058247
2,ID01AciUc_boil,-0.040439
3,ID01AciUc_pan,2.058102
4,ID0328D_boil,-0.038715
5,ID0328D_pan,2.054881
6,ID05WxObCFTs9_boil,-0.039172
7,ID05WxObCFTs9_pan,2.055782
8,ID06AdCmLMlkO_boil,0.978561
9,ID06AdCmLMlkO_pan,0.185456


In [ ]:
submission.to_csv("zindi_files/model11v02_submission11.csv", index=False)

In [ ]:
submission["Target"] = submission["Target"].apply(lambda x: round(x))
submission.to_csv("zindi_files/model11v02_submission11_round.csv", index=False)